# 1. Overview

**This notebook presents a tutorial from [Codelabs](https://codelabs.developers.google.com/?cat=Machine+Learning) for [Prototyping models in AI Platform Notebooks](https://codelabs.developers.google.com/codelabs/prototyping-caip-notebooks/index.html?index=..%2F..index#2)**

This lab will walk you through various tools in AI Platform Notebooks for exploring your data and prototyping ML models.

### What you learn

You'll learn how to:
- Create and customize an AI Platform Notebooks instance
- Track your notebooks code with git, directly integrated into AI Platform Notebooks
- Use the What-If Tool within your notebook

The total cost to run this lab on Google Cloud is about **$1**.

# 2. Setup your environment

You'll need a Google Cloud Platform project with billing enabled to run this codelab. To create a project, follow the [instructions here](https://cloud.google.com/resource-manager/docs/creating-managing-projects).

### Step 1: Enable the Cloud AI Platform Models API

Navigate to the [AI Platform Models section](https://console.cloud.google.com/ai-platform/models?folder=&organizationId=&project=poc-cit) of your Cloud Console and click Enable if it isn't already enabled.

![](https://codelabs.developers.google.com/codelabs/xgb-caip-e2e/img/d0d38662851c6af3.png)

### Step 2: Enable the Compute Engine API

Navigate to [Compute Engine](https://console.cloud.google.com/marketplace/product/google/compute.googleapis.com?project=poc-cit&folder=&organizationId=) and select **Enable** if it isn't already enabled. You'll need this to create your notebook instance.

### Step 3: Create an AI Platform Notebooks instance

Navigate to [AI Platform Notebooks section](https://console.cloud.google.com/ai-platform/notebooks/list/instances?project=poc-cit&folder=&organizationId=) of your Cloud Console and click **New Instance**. Then select the latest **TensorFlow 2 Enterprise** instance type **without GPUs**:

![](https://codelabs.developers.google.com/codelabs/prototyping-caip-notebooks/img/9f2bb4dec1ba4f4f.png)

Give your instance a name or use the default. Then we'll explore the customization options. Click the **Customize** button:

![](https://codelabs.developers.google.com/codelabs/prototyping-caip-notebooks/img/63be602ba45ca7c9.png)

AI Platform Notebooks has many different customization options, including: the region your instance is deployed in, the image type, machine size, number of GPUs, and more. We'll use the defaults for region and environment. For machine configuration, we'll use an **n1-standard-8** machine:

![](https://codelabs.developers.google.com/codelabs/prototyping-caip-notebooks/img/27101d232f765a17.png)

We won't add any GPUs, and we'll use the defaults for boot disk, networking, and permission. Select **Create** to create your instance. This will take a few minutes to complete.

### Step 4: Import Python packages

Create a new cell in the notebook and import the libraries we'll be using in this codelab:

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

# 3. Connect BigQuery data to your notebook

BigQuery, Google Cloud's big data warehouse, has made [many datasets](https://cloud.google.com/bigquery/public-data) publicly available for your exploration. AI Platform Notebooks support direct integration with BigQuery without requiring authentication.

For this lab, we'll be using the [natality dataset](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=natality&page=table&_ga=2.91160473.24786528.1580741008-413280468.1556849151). This contains data on nearly every birth in the US over a 40 year time period, including the birth weight of the child, and demographic information on the baby's parents. We'll be using a subset of the features to predict a baby's birth weight.

### Step 1: Download the BigQuery data to our notebook

We'll be using the Python client library for BigQuery to download the data into a Pandas DataFrame. The original dataset is 21GB and contains 123M rows. To keep things simple we'll only be using 10,000 rows from the dataset.

Construct the query and preview the resulting DataFrame with the following code. Here we're getting 4 features from the original dataset, along with baby weight (the thing our model will predict). The dataset goes back many years but for this model we'll use only data from after 2000:

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:407: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.063611,True,32,1,37.0
1,4.687028,True,30,3,33.0
2,7.561856,True,20,1,39.0
3,7.561856,True,31,1,37.0
4,7.312733,True,32,1,40.0


To get a summary of the numeric features in our dataset, run:

In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9997.000000,10000.000000,10000.000000,9886.000000
mean,7.303938,27.226500,1.033500,38.700890
std,1.313289,6.194646,0.192825,2.538734
min,0.595248,13.000000,1.000000,18.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.126239,32.000000,1.000000,40.000000
max,12.625874,52.000000,4.000000,47.000000


This shows the mean, standard deviation, minimum, and other metrics for our numeric columns. Finally, let's get some data on our boolean column indicating the baby's gender. We can do this with Pandas' value_counts method:

In [4]:
df['is_male'].value_counts()  # get some data on our boolean column indicating the baby's gender

True     5096
False    4904
Name: is_male, dtype: int64

Looks like the dataset is nearly balanced 50/50 by gender.

>For larger datasets, use the [BigQuery connector](https://github.com/tensorflow/io/tree/master/tensorflow_io/bigquery) instead of the Pandas integration.

# 4. Prepare the data for training

In this section, we'll divide the data into train and test sets to prepare it for training our model.

## Step 1: Extract the label column

First drop rows with null values from the dataset and shuffle the data:

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

Next, extract the label column into a separate variable and create a DataFrame with only our features. Since `is_male` is a boolean, we'll convert it to an integer so that all inputs to our model are numeric:

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

Now if you preview our dataset by running `data.head()`, you should see the four features we'll be using for training.

# 4. Initialize git

AI Platform Notebooks has a direct integration with git, so that you can do version control directly within your notebook environment. This supports committing code right in the notebook UI, or via the Terminal available in JupyterLab. In this section we'll initialize a git repository in our notebook and make our first commit via the UI.

### Step 1: Initialize a git repository

From your codelab directory, select **Git** and then **Init** from the top menu bar in JupyterLab:

![](https://codelabs.developers.google.com/codelabs/prototyping-caip-notebooks/img/d9f1729358f18e58.png)

When it asks if you want to make this directory a Git Repo, select Yes. Then select the Git icon on the left sidebar to see the status of your files and commits:

|[](https://codelabs.developers.google.com/codelabs/prototyping-caip-notebooks/img/1648d6828f11a6db.png)

### Step 2: Make your first commit

In this UI, you can add files to a commit, see file diffs (we'll get to that later), and commit your changes. Let's start by committing the notebook file we just added.

Check the box next to your `demo.ipynb` notebook file to stage it for the commit (you can ignore the `.ipynb_checkpoints/ directory`). Enter a commit message in the text box and then click on the check mark to commit your changes:

![](https://codelabs.developers.google.com/codelabs/prototyping-caip-notebooks/img/fe7366522a3a268f.png)

Enter your name and email when prompted. Then go back to the History tab to see your first commit:

|[](https://codelabs.developers.google.com/codelabs/prototyping-caip-notebooks/img/d03567c3592afb77.png)

_Note that the screenshots might not match your UI exactly, due to updates since this lab was published._